### PART I: Probability prediction
- Predict probabilities.
- Look at cross-validated performance and pick your favorite model.

In [240]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
import statsmodels.formula.api as smf
import warnings
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from statsmodels.tools.eval_measures import mse,rmse
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, Lasso
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error, r2_score
import patsy
from stargazer.stargazer import Stargazer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")



# PART I: Probability prediction

In [196]:
# read in the clean dataset
firms_df = pd.read_csv("bisnode_firms_clean.csv")

In [197]:
rawvars = ["curr_assets", "curr_liab", "extra_exp", "extra_inc", "extra_profit_loss", "fixed_assets",
              "inc_bef_tax", "intang_assets", "inventories", "liq_assets", "material_exp", "personnel_exp",
              "profit_loss_year", "sales", "share_eq", "subscribed_cap"]

qualityvars = ["balsheet_flag", "balsheet_length", "balsheet_notfullyear"]

engvar = ["total_assets_bs", "fixed_assets_bs", "liq_assets_bs", "curr_assets_bs",
            "share_eq_bs", "subscribed_cap_bs", "intang_assets_bs", "extra_exp_pl",
            "extra_inc_pl", "extra_profit_loss_pl", "inc_bef_tax_pl", "inventories_pl",
            "material_exp_pl", "profit_loss_year_pl", "personnel_exp_pl"]

engvar2 = ["extra_profit_loss_pl_quad", "inc_bef_tax_pl_quad",
             "profit_loss_year_pl_quad", "share_eq_bs_quad"]

engvar3 = []
for col in firms_df.columns:
    if col.endswith('flag_low') or col.endswith('flag_high') or col.endswith('flag_error') or col.endswith('flag_zero'):
        engvar3.append(col)

#d1 =  ["d1_sales_mil_log_mod", "d1_sales_mil_log_mod_sq",
         #"flag_low_d1_sales_mil_log", "flag_high_d1_sales_mil_log"] Data leak! Removed every d1 sales value from the code

hr = ["female", "ceo_age", "flag_high_ceo_age", "flag_low_ceo_age",
        "flag_miss_ceo_age", "ceo_count", "labor_avg_mod",
        "flag_miss_labor_avg", "foreign_management"]

In [198]:
all_vars = rawvars + qualityvars + engvar + engvar2 + engvar3 + hr 

In [199]:
firms_df[all_vars].isna().sum()

curr_assets            0
curr_liab              0
extra_exp              0
extra_inc              0
extra_profit_loss      0
                      ..
flag_miss_ceo_age      0
ceo_count              0
labor_avg_mod          0
flag_miss_labor_avg    0
foreign_management     0
Length: 74, dtype: int64

In [200]:
firms_df.dropna(inplace=True)

### Dealing with categorical variables
To avoide multicolinearity, we drop the first values

In [201]:
firms_df.head()

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,flag_high_ceo_age,flag_miss_ceo_age,ceo_young,labor_avg_mod,flag_miss_labor_avg,sales_mil_log_sq,flag_low_d1_sales_mil_log,flag_high_d1_sales_mil_log,d1_sales_mil_log_mod,d1_sales_mil_log_mod_sq
0,2013,1002029.0,2013-01-01,2013-12-31,14255.555664,217103.703125,161174.078125,0.0,0.0,0.0,...,0,0,1,0.437500,0,1.054824,0,0,-1.155013,1.334055
1,2013,1011889.0,2013-01-01,2013-12-31,66125.929688,235114.812500,16555.554688,0.0,0.0,0.0,...,0,0,0,1.583333,0,0.666460,0,0,0.019109,0.000365
2,2013,1014183.0,2013-01-01,2013-12-31,6970.370605,209562.968750,5703.703613,0.0,0.0,0.0,...,0,0,0,0.819444,0,4.632597,0,0,-0.110044,0.012110
3,2013,1022796.0,2013-01-01,2013-12-31,503.703705,3859.259277,8114.814941,0.0,0.0,0.0,...,0,0,0,0.083333,0,9.971799,0,0,0.488146,0.238287
4,2013,1035705.0,2013-01-01,2013-12-31,244.444443,2392.592529,9733.333008,0.0,0.0,0.0,...,0,0,0,0.222222,0,14.500839,0,0,-0.079375,0.006300


In [202]:
firms_df["ind2_cat"].value_counts().sort_index()

ind2_cat
26.0     735
27.0     441
28.0    1389
29.0     179
30.0     104
33.0    1382
55.0    1299
56.0    8039
Name: count, dtype: int64

In [203]:
firms_df["urban_m"].value_counts().sort_index()

urban_m
1.0    4278
2.0    3872
3.0    5418
Name: count, dtype: int64

In [204]:
firms_df["m_region_loc"].value_counts().sort_index()

m_region_loc
Central    7964
East       3404
West       2200
Name: count, dtype: int64

In [205]:
ind2_catmat = patsy.dmatrix("0 + C(ind2_cat, Treatment(reference=26))", firms_df, return_type="dataframe")
m_region_locmat = patsy.dmatrix("0 + C(m_region_loc, Treatment(reference='Central'))", firms_df, return_type="dataframe")
urban_mmat = patsy.dmatrix("0 + C(urban_m, Treatment(reference=1))", firms_df, return_type="dataframe")  

In [206]:
# Define X1
basevars = firms_df[["sales_mil_log", "sales_mil_log_sq", "profit_loss_year_pl"]]
X1 = pd.concat([basevars, ind2_catmat], axis=1)

# Define X2
X2additional_vars = firms_df[["fixed_assets_bs", "share_eq_bs","curr_liab_bs", "curr_liab_bs_flag_high", \
                          "curr_liab_bs_flag_error",  "age", "foreign_management"]]
X2 = pd.concat([X1, X2additional_vars], axis=1)

# Define X3
firm = pd.concat([firms_df[["age", "age2", "new"]], ind2_catmat, m_region_locmat, urban_mmat], axis=1)
X3 = pd.concat([firms_df[["sales_mil_log", "sales_mil_log_sq"] + engvar ], firm], axis=1)

# Define X4
X4 = pd.concat([firms_df[["sales_mil_log", "sales_mil_log_sq"] + engvar \
                                 + engvar2 + engvar3 + hr + qualityvars], firm], axis=1)

# Define X5

#Creat matrix for interactions1 variables
int1mat = patsy.dmatrix("0 + C(ind2_cat):age + C(ind2_cat):age2 \
                + C(ind2_cat):sales_mil_log + C(ind2_cat):ceo_age + C(ind2_cat):foreign_management \
                + C(ind2_cat):female + C(ind2_cat):C(urban_m) + C(ind2_cat):labor_avg_mod", 
                        firms_df, return_type="dataframe")

#Drop first level to get k-1 dummies out of k categorical levels 
for col in int1mat.columns:
    if col.startswith('C(ind2_cat)[26]') or col.endswith('C(urban_m)[1]'):
        int1mat = int1mat.drop([col], axis=1)
        
#Creat matrix for interactions2 variables        
int2mat = patsy.dmatrix("0 + sales_mil_log:age + sales_mil_log:female + sales_mil_log:profit_loss_year_pl \
                + sales_mil_log:foreign_management", 
                        firms_df, return_type="dataframe")

X5 = pd.concat([X4, int1mat, int2mat], axis=1)

# Define logitvars for LASSO
logitvars = pd.concat([X4, int1mat, int2mat], axis=1)

# Define rfvars for RF (no interactions, no modified features)
rfvars  = pd.concat([firms_df[["sales_mil"] + rawvars + hr + qualityvars], firm], axis=1)

#### Helper Functions

In [207]:
# define helper functions

def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
    
def create_coef_matrix(X, model):
    coef_matrix = pd.concat(
        [pd.DataFrame(X.columns),pd.DataFrame(model.coef_.flatten())], axis = 1
    )
    coef_matrix.columns = ['variable', 'coefficient']
    coef_matrix.iloc[-1] = ['Intercept', model.intercept_.flatten()[0]]
    return coef_matrix

def cv_summary(lambdas, C_values, model):
    d = {'lambdas': lambdas, 
         'C_values': C_values, 
         'mean_cv_score': model.scores_[1].mean(axis = 0)}
    return(pd.DataFrame(data=d))

"""def create_roc_plot(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    all_coords = pd.DataFrame({
        'fpr': fpr,
        'tpr': tpr,
        'thresholds': thresholds
    })
    
    plot = ggplot(all_coords, aes(x = 'fpr', y = 'tpr')) \
        + geom_line(color=color[0], size = 0.7) \
        + geom_area(position = 'identity', fill = 'mediumaquamarine', alpha = 0.3) \
        + xlab("False Positive Rate (1-Specifity)") \
        + ylab("True Positive Rate (Sensitivity)") \
        + geom_abline(intercept = 0, slope = 1,  linetype = "dotted", color = "black") \
        + scale_y_continuous(limits = (0, 1), breaks = seq(0, 1, .1), expand = (0, 0.01)) \
        + scale_x_continuous(limits = (0, 1), breaks = seq(0, 1, .1), expand = (0.01, 0)) \
        + theme_bw()
    return(plot)
"""

def create_roc_plot(y_true, y_pred): # this is pretty important!
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred) # on x false positive and on y true positive
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Plot ROC curve line
    ax.plot(fpr, tpr, color='k', linewidth=0.7)
    
    # Fill area under curve
    ax.fill_between(fpr, tpr, alpha=0.3, color='white')
    
    # Add diagonal dotted line
    ax.plot([0, 1], [0, 1], linestyle=':', color='black')
    
    # Set axis labels
    ax.set_xlabel('False Positive Rate (1-Specificity)')
    ax.set_ylabel('True Positive Rate (Sensitivity)')
    
    # Set axis limits and ticks
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    
    # Style similar to theme_bw()
    ax.grid(True, linestyle='-', alpha=0.2)
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_color('black')
    
    # Adjust layout
    plt.tight_layout()
    
    return fig, ax


def sigmoid_array(x):
    return(1 / (1 + np.exp(-x)))

def generate_fold_prediction(model, X, fold, param_index):
    fold_coef = model.coefs_paths_[1][fold,param_index,:]
    return(sigmoid_array(np.dot(X, np.transpose(fold_coef)[:-1]) +  np.transpose(fold_coef)[-1]))

"""def create_loss_plot(all_coords, optimal_threshold, curr_exp_loss):
    all_coords_copy = all_coords.copy()
    all_coords_copy['loss'] = (all_coords_copy.false_pos*FP + all_coords_copy.false_neg*FN)/all_coords_copy.n
    
    t = optimal_threshold
    l = curr_exp_loss
    
    plot = ggplot(all_coords_copy, aes(x = 'thresholds', y = 'loss')) + \
        geom_line(color=color[0], size=0.7) + \
        scale_x_continuous(breaks = seq(0, 1.1, by = 0.1)) + \
        coord_cartesian(xlim=(0,1))+ \
        geom_vline(xintercept = t , color = color[0] ) + \
        annotate(geom = "text", x = t - 0.01, y= max(all_coords_copy.loss) - 0.4,
                 label="best threshold: " + str(round(t,2)),
                 colour=color[1], angle=90, size = 7) +\
        annotate(geom = "text", x = t + 0.06, y= l,\
                 label= str(round(l, 2)), size = 7) +\
        theme_bw()
    return(plot)"""

def create_loss_plot(all_coords, optimal_threshold, curr_exp_loss): # what is optimal threshold here?
    # Create copy and calculate loss
    all_coords_copy = all_coords.copy()
    all_coords_copy['loss'] = (all_coords_copy.false_pos*FP + all_coords_copy.false_neg*FN)/all_coords_copy.n
    
    t = optimal_threshold
    l = curr_exp_loss

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot loss line
    ax.plot(all_coords_copy['thresholds'], all_coords_copy['loss'], 
            color= 'k', linewidth=0.7)

    # Add vertical line at optimal threshold
    ax.axvline(x=t, color = 'k')

    # Add annotations
    ax.text(t - 0.04, max(all_coords_copy.loss) - 0.5,
            f"best threshold: {t:.2f}", 
            color = 'k', 
            rotation=90, 
            fontsize = 9)
    
    ax.text(t + 0.06, l,
            f"{l:.2f}",
            fontsize = 9)

    # Set x-axis ticks and limits
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_xlim(0, 1)

    # Style similar to theme_bw()
    ax.grid(True, linestyle='-', alpha=0.2)
    ax.set_facecolor('white')
    ax.set_xlabel('threshold')
    ax.set_ylabel('loss')
    for spine in ax.spines.values():
        spine.set_color('black')

    # Adjust layout
    plt.tight_layout()

    return fig, ax



"""def create_roc_plot_with_optimal(all_coords, optimal_threshold):
    all_coords_copy = all_coords.copy()
    all_coords_copy['sp'] = all_coords_copy.true_neg/all_coords_copy.neg
    all_coords_copy['se'] = all_coords_copy.true_pos/all_coords_copy.pos
    
    best_coords = all_coords_copy[all_coords_copy.thresholds == optimal_threshold]
    sp = best_coords.sp.values[0]
    se = best_coords.se.values[0]

    plot = ggplot(all_coords_copy, aes(x = 'sp', y = 'se')) +\
        geom_line(color=color[0], size=0.7) +\
        scale_y_continuous(breaks = seq(0, 1.1, by = 0.1)) +\
        scale_x_reverse(breaks = seq(0, 1.1, by = 0.1)) +\
        geom_point(data = pd.DataFrame({'sp': [sp], 'se': [se]})) +\
        annotate(geom = "text", x = sp, y = se + 0.03,
                 label = str(round(sp, 2)) + ', ' + str(round(se, 2)), size = 7) +\
        theme_bw()
    return(plot)
"""
def create_roc_plot_with_optimal(all_coords, optimal_threshold):
    # Create copy and calculate metrics
    all_coords_copy = all_coords.copy()
    all_coords_copy['sp'] = all_coords_copy.true_neg/all_coords_copy.neg
    all_coords_copy['se'] = all_coords_copy.true_pos/all_coords_copy.pos
    
    # Get optimal point
    best_coords = all_coords_copy[all_coords_copy.thresholds == optimal_threshold]
    sp = best_coords.sp.values[0]
    se = best_coords.se.values[0]
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Plot ROC curve
    ax.plot(all_coords_copy['sp'], all_coords_copy['se'],
            color='k', linewidth=0.9)
    
    # Add optimal point
    ax.scatter([sp], [se], color='k', s = 100)
    
    # Add text annotation
    ax.text(sp, se + 0.03,
            f"{sp:.2f}, {se:.2f}",
            fontsize = 9,
            ha='center')
    ax.text(sp - 0.02, se - 0.18,
            'specificity (TNR) \n& sensitivity (TPR) \nat the best threshold',
            fontsize = 9,
            ha='center'
           )
    
    # Set axis ticks and limits
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_xlabel('specificity')
    ax.set_ylabel('sensitivity')
    
    # Reverse x-axis
    ax.set_xlim(1, 0)
    
    # Style similar to theme_bw()
    ax.grid(True, linestyle='-', alpha=0.2)
    ax.set_facecolor('white')
    for spine in ax.spines.values():
        spine.set_color('black')
    
    # Adjust layout
    plt.tight_layout()
    plt.show()
    return fig, ax

### Setting up the train and test set

In [208]:
y = firms_df["is_fast_growing"]

In [209]:
y.mean()

np.float64(0.2318691037735849)

In [210]:
X_train, X_holdout, y_train, y_holdout = train_test_split(
    firms_df,          
    y,                 
    test_size=0.2,     
    random_state=42    
)

### OLS with Cross-Validation for X1:X5

In [211]:
model_specs = {
    "OLS_Model_X1": X1,
    "OLS_Model_X2": X2, 
    "OLS_Model_X3": X3,
    "OLS_Model_X4": X4,
    "OLS_Model_X5": X5
}

results = {name: {
    'rmse_test': [], 'r2_test': [], 'pred_test_mean': [],
    'rmse_train': [], 'r2_train': [], 'pred_train_mean': []
} for name in model_specs}

k = KFold(n_splits=5, shuffle=True, random_state=42)

# Iterate through each Model 
for model_name, X_data in model_specs.items():
    
    print(f"Running Cross-Validation for: {model_name}")
    
    # Inner Loop: K-Fold Split
    for train_index, test_index in k.split(X_train):
        
        # Select data based on indices and add constant
        X_train_fold = sm.add_constant(X_data.iloc[train_index])
        X_test_fold = sm.add_constant(X_data.iloc[test_index])
        
        y_train_fold = y.iloc[train_index]
        y_test_fold = y.iloc[test_index]
        
        # Fit OLS
        model = sm.OLS(y_train_fold, X_train_fold).fit()
        
        # Predict
        y_pred_test = model.predict(X_test_fold)
        y_pred_train = model.predict(X_train_fold)
        
        # --- Store Results into the Dictionary ---
        
        # Test Metrics
        results[model_name]['rmse_test'].append(np.sqrt(mean_squared_error(y_test_fold, y_pred_test)))
        results[model_name]['r2_test'].append(r2_score(y_test_fold, y_pred_test))
        results[model_name]['pred_test_mean'].append(np.mean(y_pred_test))
        
        # Train Metrics
        results[model_name]['rmse_train'].append(np.sqrt(mean_squared_error(y_train_fold, y_pred_train)))
        results[model_name]['r2_train'].append(r2_score(y_train_fold, y_pred_train))
        results[model_name]['pred_train_mean'].append(np.mean(y_pred_train))

print("Calculation complete.")

Running Cross-Validation for: OLS_Model_X1
Running Cross-Validation for: OLS_Model_X2
Running Cross-Validation for: OLS_Model_X3
Running Cross-Validation for: OLS_Model_X4
Running Cross-Validation for: OLS_Model_X5
Calculation complete.


In [212]:
summary_rows = []
for name, metrics in results.items():
    summary_rows.append({
        'Model': name,
        'Avg_RMSE_Test': np.mean(metrics['rmse_test']),
        'Avg_R2_Test': np.mean(metrics['r2_test']),
        'Pred_Test_Mean': np.mean(metrics['pred_test_mean']),
        'Avg_RMSE_Train': np.mean(metrics['rmse_train']),
        'Avg_R2_Train': np.mean(metrics['r2_train']),
        'Pred_Train_Mean': np.mean(metrics['pred_train_mean'])

    })

summary_df = pd.DataFrame(summary_rows)
summary_df.round(4)

,Model,Avg_RMSE_Test,Avg_R2_Test,Pred_Test_Mean,Avg_RMSE_Train,Avg_R2_Train,Pred_Train_Mean
0,OLS_Model_X1,0.4182,0.0166,0.2315,0.4176,0.0195,0.2314
1,OLS_Model_X2,0.4059,0.0739,0.2316,0.4049,0.0782,0.2314
2,OLS_Model_X3,0.3893,0.1477,0.2316,0.3877,0.1549,0.2314
3,OLS_Model_X4,0.3902,0.1432,0.2309,0.3825,0.1776,0.2314
4,OLS_Model_X5,0.3891,0.1486,0.2309,0.3791,0.1920,0.2314


Model 5 performs the best.

### Logistic Regression with Cross-Validation for X1:X5

In [213]:
#This takes a while, 4min 35sec for me, but it is necessary for getting the accurate scores
train_idx = X_train.index

logit_model_vars = {
    "Logit_Model_X1": X1.loc[train_idx],
    "Logit_Model_X2": X2.loc[train_idx],
    "Logit_Model_X3": X3.loc[train_idx],
    "Logit_Model_X4": X4.loc[train_idx],
    "Logit_Model_X5": X5.loc[train_idx]
}

# Initialize storage dictionaries
logit_models = {}
CV_RMSE_folds = {}
logit_accuracy = {}
logit_auc = {}

# Loop through the dictionary
for name, X_data in logit_model_vars.items():
    print(pd.Timestamp.now(), f'Running regression for {name}...')
    
    # Setup the Brier/RMSE Model
    LRCV_brier = LogisticRegressionCV(
        Cs=[1e20],  # No regularization (Standard Logit)
        cv=k, 
        refit=True, 
        scoring='neg_brier_score', 
        solver="newton-cg", 
        tol=1e-7, 
        random_state=20250224,
        n_jobs=-1  
    )
    
    # Fit & Store RMSE 
    model = LRCV_brier.fit(X_data, y_train)
    logit_models[name] = model
    
    # Extract CV RMSE from the internal storage
    raw_scores = model.scores_[1]
    CV_RMSE_folds[name] = np.sqrt(-1 * raw_scores).flatten()
    
    # Calculate CV Accuracy 
    acc_scores = cross_val_score(model, X_data, y_train, cv=k, scoring='accuracy', n_jobs=-1)
    logit_accuracy[name] = acc_scores.mean() 
    
    # Calculate CV AUC
    auc_scores = cross_val_score(model, X_data, y_train, cv=k, scoring='roc_auc', n_jobs=-1)
    logit_auc[name] = auc_scores.mean()

2026-02-07 23:33:45.961244 Running regression for Logit_Model_X1...


/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from 

2026-02-07 23:33:49.376875 Running regression for Logit_Model_X2...


/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


2026-02-07 23:33:51.942537 Running regression for Logit_Model_X3...
2026-02-07 23:34:14.602586 Running regression for Logit_Model_X4...
2026-02-07 23:35:17.853795 Running regression for Logit_Model_X5...


In [214]:
table_rows = []

for name in logit_models.keys():
    avg_rmse = np.mean(CV_RMSE_folds[name])
    # Get the score you stored (Note: for LogisticRegression, .score() is Accuracy)
    
    table_rows.append({
        'Model': name,
        'Avg_RMSE': avg_rmse,
        'Accuracy': logit_accuracy[name],
        'AUC': logit_auc[name]
    })

# Create the DataFrame
logit_summary_table = pd.DataFrame(table_rows)
logit_summary_table = logit_summary_table.round(4)
logit_summary_table

,Model,Avg_RMSE,Accuracy,AUC
0,Logit_Model_X1,0.4180,0.7679,0.5898
1,Logit_Model_X2,0.4034,0.7710,0.6955
2,Logit_Model_X3,0.3861,0.7992,0.7477
3,Logit_Model_X4,0.3837,0.8017,0.7585
4,Logit_Model_X5,0.3823,0.8025,0.7595


The best model is Model 4 and 5 are equaly good, 4 is simpler so we prefer that.

### Lasso Logit

In [215]:
X_train_data = logitvars.loc[train_idx]
y_train_data = y_train

# Define Hyperparameters (Lambdas -> Cs)
lambdas = list(10**np.arange(-1, -4.01, -1/3))
n_obs = len(X_train_data) * 4/5 
Cs_values = [1/(l*n_obs) for l in lambdas]

#Create & Fit the RMSE-Optimized Model (Brier Score)
# We use a Pipeline to scale data INSIDE the CV loop (prevents leakage)
lasso_rmse_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', LogisticRegressionCV(
        Cs=Cs_values,
        penalty='l1',
        cv=k,                        
        scoring='neg_brier_score',  
        solver='liblinear',
        random_state=42,
        n_jobs=-1               
    ))
])

print("Running LASSO (Optimizing for RMSE)...")
lasso_rmse_pipe.fit(X_train_data, y_train_data)

# --- EXTRACTING RESULTS ---

model_internal = lasso_rmse_pipe.named_steps['lasso']

# .scores_[1] gives shape (n_folds, n_Cs). Mean across folds.
mean_brier_scores = model_internal.scores_[1].mean(axis=0)

# Find the best score (Max because it's negative brier)
best_brier_idx = np.argmax(mean_brier_scores)
best_brier_score = mean_brier_scores[best_brier_idx]
best_rmse = np.sqrt(-1 * best_brier_score)

# Get the Best Hyperparameters (C and Lambda)
best_C = model_internal.Cs_[best_brier_idx]
best_lambda_val = 1 / (best_C * n_obs)


# Calculate Accuracy an AUC
winner_model = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso_fixed', LogisticRegression(
        C=best_C,        # <--- The winner from Step 1
        penalty='l1', 
        solver='liblinear', 
        random_state=42
    ))
])

# Run 5-fold CV just for this one best configuration to get accuracy and AUC
acc_scores = cross_val_score(winner_model, X_train_data, y_train_data, cv=k, scoring='accuracy')
corresponding_accuracy = acc_scores.mean()

auc_scores = cross_val_score(winner_model, X_train_data, y_train_data, cv=k, scoring='roc_auc')
final_auc = auc_scores.mean()



Running LASSO (Optimizing for RMSE)...


In [216]:
lasso_summary_table = pd.DataFrame([{
    'Model': 'LASSO Logit',
    'Optimized_For': 'RMSE (Brier)',
    'Best_Lambda': best_lambda_val,
    'Avg_RMSE': best_rmse,
    'Accuracy': corresponding_accuracy,
    'AUC': final_auc
}])

lasso_summary_table.round(4)

,Model,Optimized_For,Best_Lambda,Avg_RMSE,Accuracy,AUC
0,LASSO Logit,RMSE (Brier),0.001,0.3817,0.8029,0.7613


In [217]:
lasso_summary_table_small = lasso_summary_table[["Model", "Avg_RMSE", "Accuracy", "AUC"]].round(4)
logit_comparison_table = pd.concat([logit_summary_table, lasso_summary_table_small], axis=0)
logit_comparison_table

,Model,Avg_RMSE,Accuracy,AUC
0,Logit_Model_X1,0.4180,0.7679,0.5898
1,Logit_Model_X2,0.4034,0.7710,0.6955
2,Logit_Model_X3,0.3861,0.7992,0.7477
3,Logit_Model_X4,0.3837,0.8017,0.7585
4,Logit_Model_X5,0.3823,0.8025,0.7595
0,LASSO Logit,0.3817,0.8029,0.7613


Logit Model 4, 5 and LASSO Logit perform almost the same, lets choose model 4 for simplicity.

In [218]:
#Keep this in the code for demonstration:)

# Access the fitted model step from your pipeline
model_step = lasso_rmse_pipe.named_steps['lasso']

# Extract coefficients
# .coef_ is a list of lists. We grab the first one [0].
coefs = model_internal.coef_[0]

# We use abs() to find the strongest drivers regardless of + or - direction
feature_importance = pd.DataFrame({
    'Feature': X_train_data.columns,
    'Coefficient': coefs,
    'Abs_Coef': np.abs(coefs)
})

# Sort by the biggest impact
top_suspects = feature_importance.sort_values(by='Abs_Coef', ascending=False).head(10)

 # Data leak found: d1 values
print(top_suspects)

                               Feature  Coefficient  Abs_Coef
1                     sales_mil_log_sq    -0.724090  0.724090
62                                 new     0.632799  0.632799
60                                 age    -0.478816  0.478816
15                 profit_loss_year_pl     0.461351  0.461351
108              C(ind2_cat)[56.0]:age    -0.333435  0.333435
16                    personnel_exp_pl    -0.285721  0.285721
61                                age2     0.280807  0.280807
159  sales_mil_log:profit_loss_year_pl     0.276504  0.276504
6                          share_eq_bs    -0.239560  0.239560
59                balsheet_notfullyear    -0.227401  0.227401


### Random Forest

In [241]:
holdout_idx = X_holdout.index
X_train_rf = rfvars.loc[train_idx]
X_holdout = rfvars.loc[holdout_idx]
y_train_rf = y_train

In [ ]:
# Define Grid & Model
grid = {
    'max_features': [5, 6, 7],
    'criterion': ['gini'],
    'min_samples_split': [11, 16]
}

prob_forest = RandomForestClassifier(
    random_state=42, 
    n_estimators=100,
    oob_score=True
)

# Run Grid Search
prob_forest_grid = GridSearchCV(
    prob_forest, 
    grid, 
    cv=5, 
    refit='accuracy',  
    scoring=['accuracy', 'roc_auc', 'neg_brier_score'], 
    n_jobs=-1
)

print("Running Random Forest Grid Search...")
prob_forest_grid.fit(X_train_rf, y_train_rf)

# --- TABLE 1: DETAILED RESULTS (Every Combination) ---

# Extract results into a DataFrame
cv_results = pd.DataFrame(prob_forest_grid.cv_results_)

# Keep only the columns we care about
# params: The hyperparams
# mean_test_...: The scores
cols_to_keep = ['param_max_features', 'param_min_samples_split', 
                'mean_test_accuracy', 'mean_test_roc_auc', 'mean_test_neg_brier_score']
summary_table = cv_results[cols_to_keep].copy()

# Rename columns for readability
summary_table.columns = ['Max_Features', 'Min_Samples_Split', 'Accuracy', 'AUC', 'Neg_Brier']

# Calculate RMSE from Neg_Brier
summary_table['RMSE'] = np.sqrt(-1 * summary_table['Neg_Brier'])

# Drop the confusing 'Neg_Brier' column now that we have RMSE
summary_table = summary_table.drop(columns=['Neg_Brier'])

# Sort by Lowest RMSE (Best Probabilities)
summary_table = summary_table.sort_values(by='RMSE', ascending=True)

print("\n--- Detailed Results (All Combinations) ---")
summary_table.round(4)


Running Random Forest Grid Search...


/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from 


--- Detailed Results (All Combinations) ---


,Max_Features,Min_Samples_Split,Accuracy,AUC,RMSE
1,5,16,0.7676,0.4855,0.4275
0,5,11,0.7672,0.4846,0.4280
3,6,16,0.7676,0.4820,0.4281
5,7,16,0.7673,0.4845,0.4282
2,6,11,0.7672,0.4823,0.4287
4,7,11,0.7672,0.4823,0.4291


In [244]:
# --- TABLE 2: BEST MODEL SUMMARY ---

# Since you used refit='accuracy', .best_estimator_ gives the model with highest accuracy.
# If you want the model with lowest RMSE, we grab the top row of our sorted table above.

best_row = summary_table.iloc[0] # The row with lowest RMSE

best_model_summary = pd.DataFrame([{
    'Model': 'Random Forest',
    'Optimized_For': 'RMSE (Manual Sort)',
    'Max_Features': best_row['Max_Features'],
    'Min_Samples_Split': best_row['Min_Samples_Split'],
    'CV_RMSE': best_row['RMSE'],
    'CV_Accuracy': best_row['Accuracy'],
    'CV_AUC': best_row['AUC']
}])

print("\n--- Best Model Summary ---")
best_model_summary.round(4)


--- Best Model Summary ---


,Model,Optimized_For,Max_Features,Min_Samples_Split,CV_RMSE,CV_Accuracy,CV_AUC
0,Random Forest,RMSE (Manual Sort),5.0,16.0,0.4275,0.7676,0.4855


### Gradient Boosting

In [246]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Setup Data
X_train_gb = rfvars.loc[train_idx] # Use the same clean vars as Random Forest
y_train_gb = y_train

# Define the Hyperparameter Grid
# Usually, lower learning_rate (0.01) + higher n_estimators (500) = Better results but slower.
gb_params = {
    'n_estimators': [100, 200],         # Number of boosting stages
    'learning_rate': [0.05, 0.1, 0.2],  # How much each tree contributes (shrinkage)
    'max_depth': [3, 5],                # Keep trees shallow! (Standard is 3-5)
    'subsample': [0.8, 1.0]             # Stochastic gradient boosting (train on random subset)
}

# Setup Model
gb_model = GradientBoostingClassifier(random_state=42)

# Run Grid Search with Multiple Metrics
gb_grid = GridSearchCV(
    gb_model, 
    gb_params, 
    cv=5, 
    refit='neg_brier_score',  # Optimize for best PROBABILITY (RMSE)
    scoring=['accuracy', 'roc_auc', 'neg_brier_score'], 
    n_jobs=-1
)

print("Running Gradient Boosting Grid Search...")
gb_grid.fit(X_train_gb, y_train_gb)

# --- TABLE 1: DETAILED RESULTS ---

cv_results_gb = pd.DataFrame(gb_grid.cv_results_)

# Select and rename columns
cols_gb = ['param_learning_rate', 'param_n_estimators', 'param_max_depth', 
           'mean_test_accuracy', 'mean_test_roc_auc', 'mean_test_neg_brier_score']

gb_summary = cv_results_gb[cols_gb].copy()
gb_summary.columns = ['Learning_Rate', 'N_Estimators', 'Max_Depth', 'Accuracy', 'AUC', 'Neg_Brier']

# Calculate RMSE
gb_summary['RMSE'] = np.sqrt(-1 * gb_summary['Neg_Brier'])
gb_summary = gb_summary.drop(columns=['Neg_Brier'])

# Sort by Best RMSE
gb_summary = gb_summary.sort_values(by='RMSE', ascending=True)

print("\n--- Gradient Boosting Results (Top 5) ---")
gb_summary.head(5).round(4)



Running Gradient Boosting Grid Search...


/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/home/ambrus2/miniconda3/envs/daenv/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from 


--- Gradient Boosting Results (Top 5) ---


,Learning_Rate,N_Estimators,Max_Depth,Accuracy,AUC,RMSE
1,0.05,100,3,0.7676,0.4815,0.4239
0,0.05,100,3,0.7676,0.4785,0.4240
5,0.05,100,5,0.7666,0.4933,0.4251
3,0.05,200,3,0.7664,0.4794,0.4253
2,0.05,200,3,0.7668,0.4751,0.4257


In [247]:
# --- TABLE 2: BEST MODEL SUMMARY ---

best_row_gb = gb_summary.iloc[0]

best_gb_summary = pd.DataFrame([{
    'Model': 'Gradient Boosting',
    'Optimized_For': 'RMSE (Brier)',
    'Learning_Rate': best_row_gb['Learning_Rate'],
    'N_Estimators': best_row_gb['N_Estimators'],
    'CV_RMSE': best_row_gb['RMSE'],
    'CV_Accuracy': best_row_gb['Accuracy'],
    'CV_AUC': best_row_gb['AUC']
}])

print("\n--- Best Gradient Boosting Model ---")
best_gb_summary.round(4)


--- Best Gradient Boosting Model ---


,Model,Optimized_For,Learning_Rate,N_Estimators,CV_RMSE,CV_Accuracy,CV_AUC
0,Gradient Boosting,RMSE (Brier),0.05,100.0,0.4239,0.7676,0.4815


# PART II: Classification

Think about the business problem, and define your loss function (like FP=X dollars, FN=Y dollars).

Idea 1: We have some spare money and want to do some investments. Overall, riskier firms, thus firms with a higher probability to default also pay higher returns. On the other hand, we lose money when a risky firm defaults. The money lost from an unexpected default is about the same, as money lost from a risky firm that ends up well performing that we decided not to invest in.
Therefore, a suggested loss function would be:
FP = 0.5 FN = 0.5

## MODELS WE CAN'T USE  :/

My dumb ass forgot that we have binary outcome variables and calculated all the regular models.

For now, I'm keeping them in the script in case that some  of the syntax might come in handy.

### LASSO

In [ ]:

# define model
model = Lasso()

grid = dict()
grid["alpha"] = np.arange(0.05, 1, 0.05)
# define search
search = GridSearchCV(model, grid, scoring="neg_root_mean_squared_error", cv = k, verbose= 3) # control your output with the 'verbose' option

In [238]:
# Initialize lists for both sets
rmse_lasso_test, r2_lasso_test = [], []
rmse_lasso_train, r2_lasso_train = [], []
pred_lasso_test, pred_lasso_train = [], []

k = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in k.split(rfvars):
    
    X_train, X_test = logitvars.iloc[train_index], logitvars.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    ### LASSO MODEL ###
    lasso_mod = search.fit(X_train, y_train)

    y_pred_test = lasso_mod.predict(X_test)
    y_pred_train = lasso_mod.predict(X_train)
    
    pred_lasso_test.append(y_pred_test.mean())
    pred_lasso_train.append(y_pred_train.mean())

    rmse_lasso_test.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))
    r2_lasso_test.append(r2_score(y_test, y_pred_test))
    
    rmse_lasso_train.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    r2_lasso_train.append(r2_score(y_train, y_pred_train))

# Quick summary of the averages
print(f"Train RMSE: {np.mean(rmse_lasso_train):.4f} vs Test RMSE: {np.mean(rmse_lasso_test):.4f}")
print(f"Train R2:   {np.mean(r2_lasso_train):.4f} vs Test R2:   {np.mean(r2_lasso_test):.4f}")

Fitting 5 folds for each of 19 candidates, totalling 95 fits
[CV 1/5] END .......................alpha=0.05;, score=-0.398 total time=   0.4s
[CV 2/5] END .......................alpha=0.05;, score=-0.393 total time=   0.3s
[CV 3/5] END .......................alpha=0.05;, score=-0.416 total time=   0.2s
[CV 4/5] END .......................alpha=0.05;, score=-0.409 total time=   0.1s
[CV 5/5] END .......................alpha=0.05;, score=-0.401 total time=   0.1s
[CV 1/5] END ........................alpha=0.1;, score=-0.399 total time=   0.1s
[CV 2/5] END ........................alpha=0.1;, score=-0.398 total time=   0.0s
[CV 3/5] END ........................alpha=0.1;, score=-0.420 total time=   0.1s
[CV 4/5] END ........................alpha=0.1;, score=-0.413 total time=   0.1s
[CV 5/5] END ........................alpha=0.1;, score=-0.405 total time=   0.1s
[CV 1/5] END ........alpha=0.15000000000000002;, score=-0.402 total time=   0.1s
[CV 2/5] END ........alpha=0.15000000000000002;,

KeyboardInterrupt: 

In [ ]:
results_lasso_mod = {
        "predicted train": pred_lasso_train,
        "r2 train": r2_lasso_train,
        "rmse train": rmse_lasso_train,
        "predicted test": pred_lasso_test,
        "r2 test": r2_lasso_test,
        "rmse test": pred_lasso_test
    }
results_lasso_mod = pd.concat([pd.DataFrame(results_lasso_mod), pd.DataFrame(pd.DataFrame(results_lasso_mod).mean(), columns=["Average"]).T])
results_lasso_mod

,predicted train,r2 train,rmse train,predicted test,r2 test,rmse test
0,0.232080,0.090186,0.402674,0.234074,0.082386,0.234074
1,0.232541,0.094465,0.402004,0.234472,0.083544,0.234472
2,0.228764,0.092679,0.400099,0.230924,0.096405,0.230924
3,0.230769,0.093049,0.401244,0.230263,0.083919,0.230263
4,0.235191,0.098702,0.402644,0.230394,0.098098,0.230394
Average,0.231869,0.093816,0.401733,0.232025,0.088870,0.232025


### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(random_state = 20250224)
tune_grid = {"max_features": [6, 8, 10, 12], "min_samples_leaf": [5, 10, 15]}

rf_random = GridSearchCV(
    estimator = rfr,
    param_grid = tune_grid,
    cv = 5,
    scoring = "neg_root_mean_squared_error",
    verbose = 3,
)
# Built into grid search, it will run on the test set, not on the train set!

In [ ]:
# Watch out, this takes 10 minutes to run!
 
rmse_rf_test, r2_rf_test = [], []
rmse_rf_train, r2_rf_train = [], []
pred_rf_test, pred_rf_train = [], []

k = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in k.split(rfvars):
    
    X_train, X_test = rfvars.iloc[train_index], rfvars.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    ### Random Forest Model ###
    rf_mod = rf_random.fit(X_train, y_train)

    y_pred_test = rf_mod.predict(X_test)
    y_pred_train = rf_mod.predict(X_train)
    
    pred_rf_test.append(y_pred_test.mean())
    pred_rf_train.append(y_pred_train.mean())

    rmse_rf_test.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))
    r2_rf_test.append(r2_score(y_test, y_pred_test))
    
    rmse_rf_train.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    r2_rf_train.append(r2_score(y_train, y_pred_train))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END max_features=6, min_samples_leaf=5;, score=-0.151 total time=   1.0s
[CV 2/5] END max_features=6, min_samples_leaf=5;, score=-0.148 total time=   0.9s
[CV 3/5] END max_features=6, min_samples_leaf=5;, score=-0.149 total time=   1.0s
[CV 4/5] END max_features=6, min_samples_leaf=5;, score=-0.146 total time=   1.0s
[CV 5/5] END max_features=6, min_samples_leaf=5;, score=-0.143 total time=   1.0s
[CV 1/5] END max_features=6, min_samples_leaf=10;, score=-0.155 total time=   0.9s
[CV 2/5] END max_features=6, min_samples_leaf=10;, score=-0.154 total time=   0.9s
[CV 3/5] END max_features=6, min_samples_leaf=10;, score=-0.155 total time=   0.9s
[CV 4/5] END max_features=6, min_samples_leaf=10;, score=-0.151 total time=   0.9s
[CV 5/5] END max_features=6, min_samples_leaf=10;, score=-0.157 total time=   0.9s
[CV 1/5] END max_features=6, min_samples_leaf=15;, score=-0.162 total time=   0.9s
[CV 2/5] END max_features=6, mi

KeyboardInterrupt: 

In [ ]:
results_rf_mod = {
        "predicted train": pred_rf_train,
        "r2 train": r2_rf_train,
        "rmse train": rmse_rf_train,
        "predicted test": pred_rf_test,
        "r2 test": r2_rf_test,
        "rmse test": pred_rf_test
    }
results_rf_mod = pd.concat([pd.DataFrame(results_rf_mod), pd.DataFrame(pd.DataFrame(results_rf_mod).mean(), columns=["Average"]).T])
results_rf_mod

### CART

In [ ]:
cart = DecisionTreeRegressor(random_state=1234, criterion="squared_error",max_depth=3)

In [ ]:
rmse_cart_test, r2_cart_test = [], []
rmse_cart_train, r2_cart_train = [], []
pred_cart_test, pred_cart_train = [], []

k = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in k.split(rfvars):
    
    X_train, X_test = rfvars.iloc[train_index], rfvars.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    ### Random Forest Model ###
    cart_mod = cart.fit(X_train, y_train)

    y_pred_test = rf_mod.predict(X_test)
    y_pred_train = rf_mod.predict(X_train)
    
    pred_cart_test.append(y_pred_test.mean())
    pred_cart_train.append(y_pred_train.mean())

    rmse_cart_test.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))
    r2_cart_test.append(r2_score(y_test, y_pred_test))
    
    rmse_cart_train.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    r2_cart_train.append(r2_score(y_train, y_pred_train))

In [ ]:
results_cart_mod = {
        "predicted train": pred_cart_train,
        "r2 train": r2_cart_train,
        "rmse train": rmse_cart_train,
        "predicted test": pred_cart_test,
        "r2 test": r2_cart_test,
        "rmse test": pred_cart_test
    }
results_cart_mod = pd.concat([pd.DataFrame(results_cart_mod), pd.DataFrame(pd.DataFrame(results_cart_mod).mean(), columns=["Average"]).T])
results_cart_mod

### BOOSTING

In [ ]:
gbm = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=20, max_features = 10
                                #, n_estimators = 50
                               )

tune_grid = {"n_estimators": [200, 300], "max_depth": [5, 10]}

gbm_model_cv = GridSearchCV(
    gbm,
    tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=10,
    n_jobs=-1
)

In [ ]:
# 1. Flatten categorical_columns and ensure no nested lists
# We use a list comprehension to make sure we only grab strings
raw_cat_list = engvar3 + ["balsheet_notfullyear", "foreign_management"]
categorical_columns = []
for item in raw_cat_list:
    if isinstance(item, list):
        categorical_columns.extend(item)
    else:
        categorical_columns.append(item)

# 2. Flatten all_vars the same way
final_all_vars = []
for item in all_vars:
    if isinstance(item, list):
        final_all_vars.extend(item)
    else:
        final_all_vars.append(item)

# 3. Filter numerical columns based on the flattened lists
numerical_columns = [col for col in final_all_vars if col not in categorical_columns]

# 4. Redefine Preprocessing
preprocessing = ColumnTransformer(
    [
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns),
        ("num", "passthrough", numerical_columns),
    ]
)

# Now try the fit again
gbm_pipe = Pipeline([("preprocess", preprocessing), ("regressor", gbm_model_cv)])

In [ ]:
# watch out this takes 10 min to run!
#
r2_gbm_test, r2_gbm_train = [], []
rmse_gbm_test, rmse_gbm_train = [], []
pred_gbm_test, pred_gbm_train = [], []

for train_index, test_index in k.split(firms_df[final_all_vars]):
    
    X_train, X_test = firms_df[final_all_vars].iloc[train_index], firms_df[final_all_vars].iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 1. Fit the model
    gbm_mod = gbm_pipe.fit(X_train, y_train)
    
    # 2. Predict for TRAIN and calculate metrics
    y_pred_train = gbm_mod.predict(X_train)  # <--- Define this!
    pred_gbm_train.append(y_pred_train.mean())
    rmse_gbm_train.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    r2_gbm_train.append(r2_score(y_train, y_pred_train))

    # 3. Predict for TEST and calculate metrics
    y_pred_test = gbm_mod.predict(X_test)    # <--- Define this!
    pred_gbm_test.append(y_pred_test.mean())
    rmse_gbm_test.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))
    r2_gbm_test.append(r2_score(y_test, y_pred_test))

In [ ]:
results_gbm_mod = {
        "predicted train": pred_gbm_train,
        "r2 train": r2_gbm_train,
        "rmse train": rmse_gbm_train,
        "predicted test": pred_gbm_test,
        "r2 test": r2_gbm_test,
        "rmse test": pred_gbm_test
    }
results_gbm_mod = pd.concat([pd.DataFrame(results_gbm_mod), pd.DataFrame(pd.DataFrame(results_gbm_mod).mean(), columns=["Average"]).T])
#pd.DataFrame(results_gbm_mod)
results_gbm_mod

### GLM model 1

In [ ]:
# Watch out, this takes 10 minutes to run!
 
rmse_glm_test, r2_glm_test = [], []
rmse_glm_train, r2_glm_train = [], []
pred_glm_test, pred_glm_train = [], []

k = KFold(n_splits=5, shuffle=True, random_state=1234)

for train_index, test_index in k.split(rfvars):
    
    X_train, X_test = rfvars.iloc[train_index], rfvars.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    ### GLM ###
    glm_modelx1 = LogisticRegression(
    solver = "newton-cg", 
    max_iter = 1000, 
    penalty = None, 
    random_state = 1234).fit(X_train, y_train)
    #regression_results(y, glm_modelx1.predict(X1))

    y_pred_test = glm_modelx1.predict(X_test)
    y_pred_train = glm_modelx1.predict(X_train)
    
    pred_glm_test.append(y_pred_test.mean())
    pred_glm_train.append(y_pred_train.mean())

    rmse_glm_test.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))
    r2_glm_test.append(r2_score(y_test, y_pred_test))
    
    rmse_glm_train.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    r2_glm_train.append(r2_score(y_train, y_pred_train))

In [ ]:
results_glm_modelx1 = {
        "predicted train": pred_glm_train,
        "r2 train": r2_glm_train,
        "rmse train": rmse_glm_train,
        "predicted test": pred_glm_test,
        "r2 test": r2_glm_test,
        "rmse test": pred_glm_test
    }
results_glm_modelx1 = pd.concat([pd.DataFrame(results_glm_modelx1), pd.DataFrame(pd.DataFrame(results_glm_modelx1).mean(), columns=["Average"]).T])
results_glm_modelx1

### comparing all models

In [ ]:
## comparing all models:

model_comparison = pd.DataFrame({'model': ['OLS', 'LASSO', "CART", 'GBM', 'RF', "GLM1"],
    'RMSE': [np.mean(rmse_modelx1_train), np.mean(rmse_lasso_train),
            np.mean(rmse_cart_train), np.mean(rmse_gbm_train), np.mean(rmse_rf_train),
            np.mean(rmse_glm_train)],
    "R2": [np.mean(r2_modelx1_train), np.mean(r2_lasso_train),
            np.mean(r2_cart_train), np.mean(r2_gbm_train), np.mean(r2_rf_train),
            np.mean(r2_glm_train)]
})

print("The Random Forest model works best in both RMSE and R2")
model_comparison